In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.model_selection import KFold
import numpy as np

In [2]:
with open('train.json') as f:
    data = json.load(f)
    
with open('test.json') as f:
    test = json.load(f)

price = data['price']
latitude = data['latitude']
longitude = data['longitude']
time = data['created']

time_t = test['created']

new_time = time
for keys in time:
    new_time[keys] = time[keys][11] + time[keys][12]

for keys in new_time:
    tmp = int(new_time[keys])
    new_time[keys] = tmp

In [3]:
new_time_t = time_t
for keys in time_t:
    new_time_t[keys] = time_t[keys][11] + time_t[keys][12]

for keys in new_time_t:
    tmp = int(new_time_t[keys])
    new_time_t[keys] = tmp
    

In [4]:
test = pd.DataFrame(test)

In [5]:
def get_bounds(variable):
    values = list(variable.values())
    values.sort()
    length = len(values)
    inedx= int((length - 1)/4)
    q1 = (values[inedx] + values[inedx + 1])/2
    inedx = int((length - 1)*3/4)
    q3 = (values[inedx] + values[inedx + 1])/2
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr            #100
    upper_bound = q3 + 1.5*iqr
    
    return lower_bound, upper_bound	

In [6]:
def dist2place(lat,lon):
    place = {'ny' : [40.78,73.97],'hells':[40.7622,73.9924],'flatiron' : [40.7413,73.9893], 'greenwich': [40.7326,74.0012],'brooklyn': [40.6960,73.9963],'prospect': [40.6779,73.9694],'noho': [40.7272,73.9934],'soho': [40.7241,74.0011], 'gramercy': [40.7362,73.9839],'murray' :[40.7486,73.9764],'financial': [40.7092,74.0106],'kips': [40.7419,73.9778],'dumbo': [40.7032,73.9884],'nolita': [40.7226,73.9952],'battery' : [40.7127,74.0161],'tribeca' : [40.718,74.008531],'gowanus': [40.6749,73.9904],'bronx': [40.8565,73.8704],'station': [40.7527,73.9760],'kennedy': [40.6457,73.7759],'newark': [40.6899,74.1737],'mroosevelt': [40.7466,73.8909],'beach': [40.5846,73.7099],'water_lat': [40.7029,73.7058]}
    lst = []
    for i in place:
        p_lat = place[i][0]
        p_lon = place[i][1]
        x = (p_lat - lat)
        y = (p_lon + lon)
        x = x*x
        y = y*y
        tmp = x+y
        dist = np.sqrt(tmp)
        lst.append(dist)
    lst =np.array(lst)
    tp = lst.transpose()
    min_lst = []
    for x in tp:
        min_dist =  min(x)
        min_lst.append(min_dist)
    return min_lst

In [7]:
data=pd.DataFrame(data)

In [8]:
filter_missing_data = data[(data['features'].map(lambda d: len(d)) != 0) & (data['latitude'].map(lambda d: d) != 0) & (data['longitude'].map(lambda d: d) != 0)]
lower_bound,upper_bound = get_bounds(price)
filter_price_outlier_data = filter_missing_data[(filter_missing_data['price'] > lower_bound) & (filter_missing_data['price'] < upper_bound)]
filter_latitude_outlier_data = filter_price_outlier_data[(filter_price_outlier_data['latitude'].map(lambda d: d >= 40))]
filter_longitude_outlier_data = filter_latitude_outlier_data[(filter_latitude_outlier_data['longitude'].map(lambda d: d >= -79.8 and d <= -73.3))]

In [9]:
data = filter_longitude_outlier_data

In [10]:
data["num_photos"] = data["photos"].apply(len)
data["num_features"] = data["features"].apply(len)
data['street_address'] = data['street_address'].astype('category').cat.codes
data['building_id']=data['building_id'].astype('category').cat.codes
data['display_address'] = data['display_address'].astype('category').cat.codes
data['manager_id']=data['manager_id'].astype('category').cat.codes
data['interest_level']=data['interest_level'].astype('category').cat.codes
data['Distance'] = dist2place(data['latitude'], data['longitude'])

In [11]:
data = data.reset_index(drop=True)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,num_photos,num_features,Distance
0,1.0,1,3410,5,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,732,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,2038,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,1680,2,12,7,0.035327
1,1.0,2,4765,5,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,4770,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,1896,[https://photos.renthop.com/2/7092344_7663c19a...,3800,4588,1,6,6,0.004050
2,1.0,2,5285,15,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,4796,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,2561,[https://photos.renthop.com/2/7158677_c897a134...,3495,8580,2,6,6,0.014328
3,1.0,0,4939,3,Over-sized Studio w abundant closets. Availabl...,4736,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,592,[https://photos.renthop.com/2/7225292_901f1984...,2795,7582,1,4,4,0.004031
4,2.0,3,326,6,AMAZING DEAL!! BRAND NEW RENOVATIONS IN THIS H...,4663,"[Fireplace, Laundry in Unit, Dishwasher, Hardw...",40.7302,7126989,-73.9826,2009,[https://photos.renthop.com/2/7126989_a74ab3da...,6000,8696,1,7,5,0.006139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43205,1.0,3,3750,3,There is 700 square feet of recently renovated...,6370,"[Elevator, Dishwasher, Hardwood Floors]",40.8433,6824800,-73.9396,2100,[https://photos.renthop.com/2/6824800_0682be16...,2800,11238,1,5,3,0.070221
43206,1.0,2,2149,2,"2 bedroom apartment with updated kitchen, rece...",3614,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",40.8198,6813268,-73.9578,1816,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,7370,2,5,7,0.041628
43207,1.0,1,2631,5,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,1177,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",40.5765,6927093,-73.9554,2038,[https://photos.renthop.com/2/6927093_93a52104...,1850,3707,2,3,11,0.102362
43208,1.0,2,1523,2,Wonderful Bright Chelsea 2 Bedroom apartment o...,7128,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",40.7448,6892816,-74.0017,2494,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,7877,2,5,5,0.012210


In [14]:
test["num_photos"] = test["photos"].apply(len)
test["num_features"] = test["features"].apply(len)
test['street_address'] = test['street_address'].astype('category').cat.codes
test['building_id']=test['building_id'].astype('category').cat.codes
test['display_address'] = test['display_address'].astype('category').cat.codes
test['manager_id']=test['manager_id'].astype('category').cat.codes
# test['interest_level']=test['interest_level'].astype('category').cat.codes
test['Distance'] = dist2place(test['latitude'], test['longitude'])

In [15]:
test = test.reset_index(drop=True)
test

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,num_photos,num_features,Distance
0,1.0,1,4412,5,Large with awesome terrace--accessible via bed...,9506,"[Elevator, Laundry in Building, Laundry in Uni...",40.7185,7142618,-73.9865,2694,[https://photos.renthop.com/2/7142618_1c45a2c8...,2950,18989,8,6,0.009618
1,1.0,2,0,6,Prime Soho - between Bleecker and Houston - Ne...,9589,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7210040,-74.0000,3145,[https://photos.renthop.com/2/7210040_d824cc71...,2850,4161,3,3,0.003860
2,1.0,0,0,1,Spacious studio in Prime Location. Cleanbuildi...,9513,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7260,7174566,-74.0026,3467,[https://photos.renthop.com/2/7174566_ba3a35c5...,2295,1151,1,3,0.002421
3,1.0,2,9082,5,For immediate access call Bryan.<br /><br />Bo...,8437,"[Hardwood Floors, Dogs Allowed, Cats Allowed]",40.7321,7191391,-74.0028,1007,[https://photos.renthop.com/2/7191391_8c2f2d49...,2900,6746,4,3,0.001676
4,1.0,1,4680,7,Beautiful TRUE 1 bedroom in a luxury building ...,8016,"[Roof Deck, Doorman, Elevator, Fitness Center,...",40.7054,7171695,-74.0095,2521,[https://photos.renthop.com/2/7171695_089ffee2...,3254,4816,6,10,0.003956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74654,1.0,1,6889,16,BRAND NEW TO MARKET 1BDR \r107TH AND LEXINGTON...,1336,[],40.7925,6928108,-73.9454,1068,[https://photos.renthop.com/2/6928108_231eb983...,1700,3248,10,0,0.027594
74655,1.0,2,5268,5,Convertible 2BR apartment features a brand new...,6755,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7456,6906674,-73.9797,660,[https://photos.renthop.com/2/6906674_9fe899a8...,4195,2324,4,8,0.004159
74656,1.0,0,0,1,"Let's get you in to see this $2,400/mo, recent...",8565,"[Dogs Allowed, Cats Allowed]",40.7416,6897967,-73.9829,3467,[],2400,18762,0,2,0.005109
74657,2.0,2,7301,2,CooperCooper.com :: Web ID #171357; Access 100...,9037,"[Doorman, Elevator, Cats Allowed, Dogs Allowed]",40.7485,6842183,-73.9800,1653,[https://photos.renthop.com/2/6842183_b1fe51f4...,6895,12788,8,4,0.003601


In [16]:
feature_name = ['bathrooms',  'bedrooms','created', 'display_address', 'latitude', 'longitude', 'manager_id','price', 'num_photos', 'num_features','Distance']
x = data[feature_name]
y=data.interest_level

In [17]:
dtc = DecisionTreeClassifier(criterion='gini',max_depth = 6)
dtc.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
X_test = test[feature_name]

In [23]:
y_pred = dtc.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ["high" ,"low","medium"]

In [24]:
sub = pd.DataFrame([])

In [25]:
sub['listing_id'] = test['listing_id']
sub['high'] = y_pred['high']
sub['medium'] = y_pred['medium']
sub['low'] = y_pred['low']

In [26]:
sub.to_csv('dtc_test',index=False)

In [ ]:
dtc =  LogisticRegression(penalty = 'l2', solver = 'newton-cg', multi_class ='multinomial', max_iter = 10000)
dtc.fit(x,y)
X_test = test[feature_name]
y_pred = dtc.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ["high" ,"low","medium"]
sub = pd.DataFrame([])
sub['listing_id'] = test['listing_id']
sub['high'] = y_pred['high']
sub['medium'] = y_pred['medium']
sub['low'] = y_pred['low']
sub.to_csv('dtc_test',index=False)

In [41]:
feature_name = [ 'display_address', 'price']
x = data[feature_name]
y=data.interest_level
X_test = test[feature_name]

In [ ]:
dtc =  SVC(kernel='linear',probability=True)
dtc.fit(x,y)
X_test = test[feature_name]
y_pred = dtc.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ["high" ,"low","medium"]
sub = pd.DataFrame([])
sub['listing_id'] = test['listing_id']
sub['high'] = y_pred['high']
sub['medium'] = y_pred['medium']
sub['low'] = y_pred['low']
sub.to_csv('dtc_test',index=False)